# Build dataframe
## Sabrina Guastavino, guastavino@dima.unige.it

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import csv
import random
import pandas
import pickle
import os.path
import sys
import matplotlib
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
import glob
import time
import os.path

from sklearn.metrics import confusion_matrix

import keras
import numpy
from keras.layers import Dense, Flatten, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam

from keras.models import Model
from keras.models import Input


from tensorflow.keras import regularizers


from keras.layers import Activation
from sklearn.metrics import confusion_matrix

from keras import backend as K
import tensorflow as tf


from collections import deque
import sys

from keras.regularizers import l2

In [2]:
from utilities.utilities_solar_flare_forecasting import training_set_standardization, testing_set_standardization, compute_cm_tss,\
compute_weight_cm_tss_harp, optimize_time_weighted_tss_solar_flares,classification_skills_weight_solar_flares_threshold,\
optimize_tss,compute_cm_tss_threshold, choose_thresholds_tss_wtss, construct_panel_info_train, predict_ensemble,\
read_data_construct_sets,select_best_patience_on_val, predict

## Experiments prediction C1+ flares

### Read data and construct training, validation and test sets

In [3]:
# read panel features and label for training and validation sets
folder_data='data/'
class_flare = 'abovec'

X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                   class_flare)


#samples in training:  3840
#yes in training:  1027
#samples in validation:  896
#yes in validation:  147


### Procedure to select thresholds with TSS and wTSS optimization strategies

In [4]:

#Choose thresholds for converting the probability outcome in 0/1 by optimizing the TSS and wTSS on the training set
#class C

folder = 'prediction/checkpoints/'

list_epochs = sorted(glob.glob(folder+'nn7593param_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01*.hdf5'))#'lstm_1times_pred_poll_6features_train_13104_val_4320_poll_300_epoch*.hdf5'))
file_name = 'nn7593param_new_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01.npy'
folder_to_save = 'utilities/save_variables/'

run = False
save = False

if run == True:
    opt_threshold, opt_threshold_weight, tss, tss_weight, tss_val_array, tss_val_weight_array = choose_thresholds_tss_wtss(folder,list_epochs,file_name,X_train_std,Y_train_new, X_val_std,Y_val,\
                                                                                                                       df_train,panel_23features_val_from_2015_09_29_to_2015_11_01)
    #save variables
    if save == True:
        numpy.save(folder_to_save+'opt_threshold_'+file_name+'.npy',opt_threshold) 
        numpy.save(folder_to_save+'opt_threshold_weight_'+file_name+'.npy',opt_threshold_weight) 
        numpy.save(folder_to_save+'tss_train_'+file_name+'.npy',tss) 
        numpy.save(folder_to_save+'tss_train_weight_'+file_name+'.npy',tss_weight)
        numpy.save(folder_to_save+'tss_val_'+file_name+'.npy',tss_val_array) 
        numpy.save(folder_to_save+'tss_val_weight_'+file_name+'.npy',tss_val_weight_array)

### Predict on the test set by using the ensemble method with TSS and wTSS optimization strategies

In [5]:
#class C
#predict on test
folder = 'prediction/checkpoints/'


list_epochs = sorted(glob.glob(folder+'nn7593param_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01*.hdf5'))#'lstm_1times_pred_poll_6features_train_13104_val_4320_poll_300_epoch*.hdf5'))
file_name = 'nn7593param_new_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01'


folder_to_save = 'utilities/save_variables/'

#load variables
opt_threshold=numpy.load(folder_to_save+'opt_threshold_'+file_name+'.npy') 
opt_threshold_weight=numpy.load(folder_to_save+'opt_threshold_weight_'+file_name+'.npy') 
tss=numpy.load(folder_to_save+'tss_train_'+file_name+'.npy') 
tss_weight=numpy.load(folder_to_save+'tss_train_weight_'+file_name+'.npy')
tss_val_array=numpy.load(folder_to_save+'tss_val_'+file_name+'.npy') 
tss_val_weight_array=numpy.load(folder_to_save+'tss_val_weight_'+file_name+'.npy')

perc=0.9

print('C1+ flares forecasting')

print('Ensemble with TSS optimization strategy')
#prediction on the test set with the tss optimization strategy
pred_median_pred_0_1 = predict_ensemble(tss_val_array,perc,list_epochs,X_test_std,opt_threshold)

print('Results on the test set: ensemble with TSS optimization strategy')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, pred_median_pred_0_1)#_weight

print('Skill scores (optimization tss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, pred_median_pred_0_1,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization tss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))


print('Ensemble with wTSS optimization strategy')
#prediction on the test set with the wtss optimization strategy
pred_median_pred_0_1_weight = predict_ensemble(tss_val_weight_array,perc,list_epochs,X_test_std,opt_threshold_weight)


print('Results on the test set: ensemble with wTSS optimization strategy')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, pred_median_pred_0_1_weight)#_weight

print('Skill scores (optimization wtss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))


wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, pred_median_pred_0_1_weight,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization wtss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))


C1+ flares forecasting
Ensemble with TSS optimization strategy
Results on the test set: ensemble with TSS optimization strategy
Skill scores (optimization tss strategy)
[[198  30]
 [  3  31]]
tss =  0.7802
hss =  0.5832
csi =  0.4844
Weighted Skill scores (optimization tss strategy)
[[198.          29.91666667]
 [  2.5         31.        ]]
wtss =  0.7941
whss =  0.5886
wcsi =  0.4888
Ensemble with wTSS optimization strategy
Results on the test set: ensemble with wTSS optimization strategy
Skill scores (optimization wtss strategy)
[[196  32]
 [  2  32]]
tss =  0.8008
hss =  0.5823
csi =  0.4848
Weighted Skill scores (optimization wtss strategy)
[[196.          33.91666667]
 [  1.5         32.        ]]
wtss =  0.8077
whss =  0.5715
wcsi =  0.4747


## Experiments prediction M1+ flares

### Read data and construct training, validation and test sets

In [6]:
# read panel features and label for training and validation sets
folder_data='data/'
class_flare = 'abovem'

X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                   class_flare)


#samples in training:  3840
#yes in training:  168
#samples in validation:  896
#yes in validation:  15


### Procedure to select thresholds with TSS and wTSS optimization strategies

In [7]:

#Choose thresholds for converting the pprobability outcome in 0/1 by optimizing the TSS and wTSS on the training set
#

folder = 'prediction/checkpoints/'

list_epochs = sorted(glob.glob(folder+'nn7593param_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01*.hdf5'))#'lstm_1times_pred_poll_6features_train_13104_val_4320_poll_300_epoch*.hdf5'))
file_name = 'nn7593param_new_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01.npy'
folder_to_save = 'utilities/save_variables/'

run = False
save = False

if run == True:
    opt_threshold, opt_threshold_weight, tss, tss_weight, tss_val_array, tss_val_weight_array = choose_thresholds_tss_wtss(folder,list_epochs,file_name,X_train_std,Y_train_new, X_val_std,Y_val,\
                                                                                                                      df_train,panel_23features_val_from_2015_09_29_to_2015_11_01)
    if save == True:
        #save variables
        numpy.save(folder_to_save+'opt_threshold_'+file_name+'.npy',opt_threshold) 
        numpy.save(folder_to_save+'opt_threshold_weight_'+file_name+'.npy',opt_threshold_weight) 
        numpy.save(folder_to_save+'tss_train_'+file_name+'.npy',tss) 
        numpy.save(folder_to_save+'tss_train_weight_'+file_name+'.npy',tss_weight)
        numpy.save(folder_to_save+'tss_val_'+file_name+'.npy',tss_val_array) 
        numpy.save(folder_to_save+'tss_val_weight_'+file_name+'.npy',tss_val_weight_array)

### Predict on the test set by using the ensemble method with TSS and wTSS optimization strategies

In [8]:
#class M
#predict on test
folder = 'prediction/checkpoints/'


list_epochs = sorted(glob.glob(folder+'nn7593param_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01*.hdf5'))#'lstm_1times_pred_poll_6features_train_13104_val_4320_poll_300_epoch*.hdf5'))
file_name = 'nn7593param_new_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01'


folder_to_save = 'utilities/save_variables/'

#load variables
opt_threshold=numpy.load(folder_to_save+'opt_threshold_'+file_name+'.npy') 
opt_threshold_weight=numpy.load(folder_to_save+'opt_threshold_weight_'+file_name+'.npy') 
tss=numpy.load(folder_to_save+'tss_train_'+file_name+'.npy') 
tss_weight=numpy.load(folder_to_save+'tss_train_weight_'+file_name+'.npy')
tss_val_array=numpy.load(folder_to_save+'tss_val_'+file_name+'.npy') 
tss_val_weight_array=numpy.load(folder_to_save+'tss_val_weight_'+file_name+'.npy')


perc=0.95

print('M1+ flares forecasting')

print('Ensemble with TSS optimization strategy')
#prediction on the test set with the tss optimization strategy
pred_median_pred_0_1 = predict_ensemble(tss_val_array,perc,list_epochs,X_test_std,opt_threshold)

print('Results on the test set: ensemble with TSS optimization strategy')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, pred_median_pred_0_1)#_weight

print('Skill scores (optimization tss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, pred_median_pred_0_1,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization tss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))


print('Ensemble with wTSS optimization strategy')
#prediction on the test set with the wtss optimization strategy
pred_median_pred_0_1_weight = predict_ensemble(tss_val_weight_array,perc,list_epochs,X_test_std,opt_threshold_weight)


print('Results on the test set: ensemble with wTSS optimization strategy')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, pred_median_pred_0_1_weight)#_weight

print('Skill scores (optimization wtss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))


wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, pred_median_pred_0_1_weight,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization wtss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

M1+ flares forecasting
Ensemble with TSS optimization strategy
Results on the test set: ensemble with TSS optimization strategy
Skill scores (optimization tss strategy)
[[235  21]
 [  1   5]]
tss =  0.7513
hss =  0.2859
csi =  0.1852
Weighted Skill scores (optimization tss strategy)
[[235.    36.25]
 [  1.     5.  ]]
wtss =  0.6997
whss =  0.1807
wcsi =  0.1183
Ensemble with wTSS optimization strategy
Results on the test set: ensemble with wTSS optimization strategy
Skill scores (optimization wtss strategy)
[[237  19]
 [  1   5]]
tss =  0.7591
hss =  0.3080
csi =  0.2000
Weighted Skill scores (optimization wtss strategy)
[[237.    32.25]
 [  1.     5.  ]]
wtss =  0.7136
whss =  0.2012
wcsi =  0.1307


## Experiments prediction X1+ flares

### Read data and construct training, validation and test sets

In [9]:
# read panel features and label for training and validation sets
folder_data='data/'
class_flare = 'abovex'

X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                   class_flare)


#samples in training:  2560
#yes in training:  6
#samples in validation:  896
#yes in validation:  3


### Procedure to select thresholds with TSS and wTSS optimization strategies

In [10]:

#Choose thresholds for converting the pprobability outcome in 0/1 by optimizing the TSS and wTSS on the training set
#

folder = 'prediction/checkpoints/'

list_epochs = sorted(glob.glob(folder+'nn7593param_'+class_flare+'_00_std_reg2layers_false_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01*.hdf5'))#'lstm_1times_pred_poll_6features_train_13104_val_4320_poll_300_epoch*.hdf5'))
file_name = 'nn7593param_'+class_flare+'_00_std_std_reg2layers_false_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01.npy'
folder_to_save = 'utilities/save_variables/'

if run == True:
    opt_threshold, opt_threshold_weight, tss, tss_weight, tss_val_array, tss_val_weight_array = choose_thresholds_tss_wtss(folder,list_epochs,file_name,X_train_std,Y_train_new, X_val_std,Y_val,\
                                                                                                                      df_train,panel_23features_val_from_2015_09_29_to_2015_11_01)
    if save == True: 
        #save variables
        numpy.save(folder_to_save+'opt_threshold_'+file_name+'.npy',opt_threshold) 
        numpy.save(folder_to_save+'opt_threshold_weight_'+file_name+'.npy',opt_threshold_weight) 
        numpy.save(folder_to_save+'tss_train_'+file_name+'.npy',tss) 
        numpy.save(folder_to_save+'tss_train_weight_'+file_name+'.npy',tss_weight)
        numpy.save(folder_to_save+'tss_val_'+file_name+'.npy',tss_val_array) 
        numpy.save(folder_to_save+'tss_val_weight_'+file_name+'.npy',tss_val_weight_array)

### Predict on the test set by using the ensemble method with TSS and wTSS optimization strategies

In [11]:
#class X
#predict on test
folder = 'prediction/checkpoints/'


list_epochs = sorted(glob.glob(folder+'nn7593param_'+class_flare+'_00_std_reg2layers_false_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01*.hdf5'))#'lstm_1times_pred_poll_6features_train_13104_val_4320_poll_300_epoch*.hdf5'))
file_name = 'nn7593param_'+class_flare+'_00_std_reg2layers_false_train_from_2012_09_15_to_2015_10_02_val_from_2015_09_29_to_2015_11_01'


folder_to_save = 'utilities/save_variables/'

#load variables
opt_threshold=numpy.load(folder_to_save+'opt_threshold_'+file_name+'.npy') 
opt_threshold_weight=numpy.load(folder_to_save+'opt_threshold_weight_'+file_name+'.npy') 
tss=numpy.load(folder_to_save+'tss_train_'+file_name+'.npy') 
tss_weight=numpy.load(folder_to_save+'tss_train_weight_'+file_name+'.npy')
tss_val_array=numpy.load(folder_to_save+'tss_val_'+file_name+'.npy') 
tss_val_weight_array=numpy.load(folder_to_save+'tss_val_weight_'+file_name+'.npy')

perc=0.8



print('X1+ flares forecasting')


print('Ensemble with TSS optimization strategy')
#prediction on the test set with the tss optimization strategy
pred_median_pred_0_1 = predict_ensemble(tss_val_array,perc,list_epochs,X_test_std,opt_threshold)

print('Results on the test set: ensemble with TSS optimization strategy')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, pred_median_pred_0_1)#_weight

print('Skill scores (optimization tss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, pred_median_pred_0_1,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization tss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))


print('Ensemble with wTSS optimization strategy')
#prediction on the test set with the wtss optimization strategy
pred_median_pred_0_1_weight = predict_ensemble(tss_val_weight_array,perc,list_epochs,X_test_std,opt_threshold_weight)


print('Results on the test set: ensemble with wTSS optimization strategy')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, pred_median_pred_0_1_weight)#_weight

print('Skill scores (optimization wtss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))


wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, pred_median_pred_0_1_weight,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization wtss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

X1+ flares forecasting
Ensemble with TSS optimization strategy
Results on the test set: ensemble with TSS optimization strategy
Skill scores (optimization tss strategy)
[[229  31]
 [  0   2]]
tss =  0.8808
hss =  0.1013
csi =  0.0606
Weighted Skill scores (optimization tss strategy)
[[229.   60.5]
 [  0.    2. ]]
wtss =  0.7910
whss =  0.0494
wcsi =  0.0320
Ensemble with wTSS optimization strategy
Results on the test set: ensemble with wTSS optimization strategy
Skill scores (optimization wtss strategy)
[[240  20]
 [  0   2]]
tss =  0.9231
hss =  0.1548
csi =  0.0909
Weighted Skill scores (optimization wtss strategy)
[[240.  40.]
 [  0.   2.]]
wtss =  0.8571
whss =  0.0784
wcsi =  0.0476


### Early stopping on solar flare forecasting

In [12]:
## Model and fit for early stopping experiments
# Example with early stopping
if run == True:
    # read panel features and label for training and validation sets
    folder_data='data/'
    class_flare = 'abovex'

    X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                       class_flare)

    folder_path='prediction/'
    model = Sequential()

    model.add(Dense(64, input_dim=23,  kernel_regularizer=l2(0.01), activation='relu'))
    model.add(Dense(48,  kernel_regularizer=l2(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))#
    model.add(Dense(24,  activation='relu'))
    model.add(Dense(16,  activation='relu'))
    model.add(Dense(8,   activation='relu'))
    model.add(Dense(4,  activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=1e-3)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    model.summary()
    # fit network

    #choose the patience (experiments with 10,20,30,40,50)
    patience=40
    callback = keras.callbacks.EarlyStopping(monitor='val_loss',\
                            patience=patience,mode='min')

    model.fit(X_train_std, Y_train_new, epochs=200, batch_size=64, validation_data=(X_val_std, Y_val), 
                        callbacks=[callback],verbose=2, shuffle=False)

    if save == True:
        model.save_weights(folder_path+'early_stop'+str(patience)+'_nn7593param_'+class_flare+'_00_std_reg2layers_train_from_2012_09_15_to_2014_10_09_val_from_2014_10_10_to_2015_06_14.hdf5')

    ep = len(model.history.history['loss'])
    print('epochs:',ep)
    # plot history
    plt.plot(model.history.history['loss'], label='train')
    plt.plot(model.history.history['val_loss'], label='val')
    plt.legend()
    plt.show()

### C1+ flares  prediction with early stopping

In [13]:
# read panel features and label for training and validation sets
folder_data='data/'
class_flare = 'abovec'
folder_path = 'prediction/'
X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                   class_flare)

tss_opt_tss,wtss_opt_wtss, threshold_opt_tss, threshold_opt_wtss=select_best_patience_on_val(folder_path,class_flare,X_train_std,Y_train_new, df_train, X_val_std, Y_val,panel_23features_val_from_2015_09_29_to_2015_11_01)



#samples in training:  3840
#yes in training:  1027
#samples in validation:  896
#yes in validation:  147
A: 0.95170015
B: 0.006513443
MAX TSS: 0.5784918520576227
idx best tss= (array([25]),)
best TSS
confusion matrix
[[2370.          470.41666667]
 [ 276.83333333  805.        ]]
false alarm ratio       	 0.36883371447239477
probability of detection	 0.7441072253890002
accuracy                	 0.8094843520938237
hss                     	 0.5481290843635969
tss                     	 0.5784918520576227
balance                 	 0.5884853852967223
csi                 	 0.5186020293122885
[0.24281013]
start optimization tss----
A: 0.95170015
B: 0.006513443
MAX TSS: 0.6263533026347625
idx best tss= (array([25]),)
best TSS
confusion matrix
[[2370, 443], [222, 805]]
false alarm ratio       	 0.35496794871794873
probability of detection	 0.7838364167478091
accuracy                	 0.8268229166666666
hss                     	 0.5863012846433793
tss                     	 0.6263533026347625
bal

In [14]:
#predict on test set

idx_patience_tss = numpy.argmax(tss_opt_tss)
idx_patience_wtss = numpy.argmax(wtss_opt_wtss)


y_pred_test = predict(folder_path,idx_patience_tss,class_flare,X_test_std,threshold_opt_tss)

print('C1+ flares forecasting')
print('Results on test set obatined with early stopping and TSS optimization :')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, y_pred_test)#_weight
print('Skill scores (optimization tss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, y_pred_test,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization tss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

y_pred_test_weight = predict(folder_path,idx_patience_wtss,class_flare,X_test_std,threshold_opt_wtss)

print('Results on test set obatined with early stopping and wTSS optimization :')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, y_pred_test_weight)
print('Skill scores (optimization wtss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, y_pred_test_weight,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization wtss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

C1+ flares forecasting
Results on test set obatined with early stopping and TSS optimization :
Skill scores (optimization tss)
[[199  29]
 [  6  28]]
tss =  0.6963
hss =  0.5407
csi =  0.4444
Weighted Skill scores (optimization tss)
[[199.    31.75]
 [  6.    28.  ]]
wtss =  0.6859
whss =  0.5185
wcsi =  0.4259
Results on test set obatined with early stopping and wTSS optimization :
Skill scores (optimization wtss)
[[193  35]
 [  2  32]]
tss =  0.7877
hss =  0.5575
csi =  0.4638
Weighted Skill scores (optimization wtss)
[[193.          42.41666667]
 [  1.66666667  32.        ]]
wtss =  0.7703
whss =  0.5072
wcsi =  0.4206


### M1+ flares  prediction with early stopping

In [15]:
# read panel features and label for training and validation sets
folder_data='data/'
class_flare = 'abovem'
folder_path = 'prediction/'
X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                   class_flare)

tss_opt_tss,wtss_opt_wtss, threshold_opt_tss, threshold_opt_wtss=select_best_patience_on_val(folder_path,class_flare,X_train_std,Y_train_new, df_train, X_val_std, Y_val,panel_23features_val_from_2015_09_29_to_2015_11_01)



#samples in training:  3840
#yes in training:  168
#samples in validation:  896
#yes in validation:  15
A: 0.5091005
B: 4.391479e-05
MAX TSS: 0.5945897195988972
idx best tss= (array([13]),)
best TSS
confusion matrix
[[3103.          889.        ]
 [  32.41666667  145.        ]]
false alarm ratio       	 0.8597678916827852
probability of detection	 0.8172851103804603
accuracy                	 0.779005856134951
hss                     	 0.17981054569127267
tss                     	 0.5945897195988972
balance                 	 0.036464191976003014
csi                 	 0.13596936782058297
[0.06622127]
start optimization tss----
A: 0.5091005
B: 4.391479e-05
MAX TSS: 0.7086056644880174
idx best tss= (array([12]),)
best TSS
confusion matrix
[[3061, 611], [21, 147]]
false alarm ratio       	 0.8060686015831134
probability of detection	 0.875
accuracy                	 0.8354166666666667
hss                     	 0.2648384563934045
tss                     	 0.7086056644880174
balance           

In [16]:
#predict on test set

idx_patience_tss = numpy.argmax(tss_opt_tss)
idx_patience_wtss = numpy.argmax(wtss_opt_wtss)


y_pred_test = predict(folder_path,idx_patience_tss,class_flare,X_test_std,threshold_opt_tss)

print('M1+ flares forecasting')
print('Results on test set obatined with early stopping and TSS optimization :')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, y_pred_test)#_weight
print('Skill scores (optimization tss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, y_pred_test,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization tss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

y_pred_test_weight = predict(folder_path,idx_patience_wtss,class_flare,X_test_std,threshold_opt_wtss)

print('Results on test set obatined with early stopping and wTSS optimization :')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, y_pred_test_weight)
print('Skill scores (optimization wtss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, y_pred_test_weight,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization wtss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

M1+ flares forecasting
Results on test set obatined with early stopping and TSS optimization :
Skill scores (optimization tss)
[[232  24]
 [  2   4]]
tss =  0.5729
hss =  0.2053
csi =  0.1333
Weighted Skill scores (optimization tss)
[[232.    44.75]
 [  1.75   4.  ]]
wtss =  0.5340
whss =  0.1145
wcsi =  0.0792
Results on test set obatined with early stopping and wTSS optimization :
Skill scores (optimization wtss)
[[234  22]
 [  2   4]]
tss =  0.5807
hss =  0.2210
csi =  0.1429
Weighted Skill scores (optimization wtss)
[[234.    40.75]
 [  1.75   4.  ]]
wtss =  0.5473
whss =  0.1267
wcsi =  0.0860


### X1+ flares  prediction with early stopping

In [17]:
# read panel features and label for training and validation sets
folder_data='data/'
class_flare = 'abovex'
folder_path = 'prediction/'
X_train_std,Y_train_new, X_val_std, Y_val, df_train, panel_23features_val_from_2015_09_29_to_2015_11_01, X_test_std, Y_test, panel_test_2017_until_harp5634 = read_data_construct_sets(folder_data,\
                                                                                                                                   class_flare)

tss_opt_tss,wtss_opt_wtss, threshold_opt_tss, threshold_opt_wtss=select_best_patience_on_val(folder_path,class_flare,X_train_std,Y_train_new, df_train, X_val_std, Y_val,panel_23features_val_from_2015_09_29_to_2015_11_01)



#samples in training:  2560
#yes in training:  6
#samples in validation:  896
#yes in validation:  3
A: 0.14879087
B: 4.7477108e-05
MAX TSS: 0.7180815131727089
idx best tss= (array([1]),)
best TSS
confusion matrix
[[2126.          834.66666667]
 [   0.            6.        ]]
false alarm ratio       	 0.992862807295797
probability of detection	 1.0
accuracy                	 0.7186516853932584
hss                     	 0.010197907121266657
tss                     	 0.7180815131727089
balance                 	 0.0
csi                 	 0.007137192704203014
[0.00153491]
start optimization tss----
A: 0.14879087
B: 4.7477108e-05
MAX TSS: 0.8324197337509789
idx best tss= (array([1]),)
best TSS
confusion matrix
[[2126, 428], [0, 6]]
false alarm ratio       	 0.9861751152073732
probability of detection	 1.0
accuracy                	 0.8328125
hss                     	 0.022754354294358146
tss                     	 0.8324197337509789
balance                 	 0.0
[0.00153491]
A: 0.16638939
B: 1

In [18]:
#predict on test set

idx_patience_tss = numpy.argmax(tss_opt_tss)
idx_patience_wtss = numpy.argmax(wtss_opt_wtss)


y_pred_test = predict(folder_path,idx_patience_tss,class_flare,X_test_std,threshold_opt_tss)

print('X1+ flares forecasting')
print('Results on test set obatined with early stopping and TSS optimization :')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, y_pred_test)#_weight
print('Skill scores (optimization tss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, y_pred_test,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization tss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

y_pred_test_weight = predict(folder_path,idx_patience_wtss,class_flare,X_test_std,threshold_opt_wtss)

print('Results on test set obatined with early stopping and wTSS optimization :')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(Y_test, y_pred_test_weight)
print('Skill scores (optimization wtss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss_harp(Y_test, y_pred_test_weight,panel_test_2017_until_harp5634)#_weight
print('Weighted Skill scores (optimization wtss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

X1+ flares forecasting
Results on test set obatined with early stopping and TSS optimization :
Skill scores (optimization tss)
[[237  23]
 [  0   2]]
tss =  0.9115
hss =  0.1359
csi =  0.0800
Weighted Skill scores (optimization tss)
[[237.   44.5]
 [  0.    2. ]]
wtss =  0.8419
whss =  0.0699
wcsi =  0.0430
Results on test set obatined with early stopping and wTSS optimization :
Skill scores (optimization wtss)
[[237  23]
 [  0   2]]
tss =  0.9115
hss =  0.1359
csi =  0.0800
Weighted Skill scores (optimization wtss)
[[237.   44.5]
 [  0.    2. ]]
wtss =  0.8419
whss =  0.0699
wcsi =  0.0430
